In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
import pathlib
import pickle
from pathlib import Path
from skimage.io import imread
import seaborn as sns
sns.set_style('white')
sys.setrecursionlimit(10000)
%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel
from cellbgnet.utils.plot_funcs import plot_od, plot_train_record

In [4]:
model_path = Path('/mnt/sda1/SMLAT/training_runs/model_rotated_45_venus25nm_dec19_700range_10k.pkl')

In [5]:
with open(model_path, 'rb') as f:
    chromo_model = pickle.load(f)

In [6]:
chromo_model.evaluation_params['eval_imgs'].shape

(30, 1041, 1302)

In [7]:
plot_train_record(chromo_model)

#### Photon count range, get background average from simulation using model paramteres

In [8]:
mol_photons = (chromo_model.data_generator.simulation_params['min_photon'] + 1) /2 * chromo_model.data_generator.psf_params['photon_scale']

In [9]:
mol_photons

1687.5

#### Plot network outputs for the eval images after tiling and re-tiling of one eval image

In [10]:
from cellbgnet.analyze_eval import recognition, plot_full_img_predictions, assemble_full_img_predictions

In [11]:
eval_img = chromo_model.evaluation_params['eval_imgs']

In [12]:
eval_img.shape

(30, 1041, 1302)

In [13]:
plt.figure()
plt.imshow(eval_img[0], cmap='gray')
plt.colorbar()
plt.show()

In [14]:
fov_size = [eval_img.shape[2] * 65, eval_img.shape[1] * 65]

In [15]:
fov_size

[84630, 67665]

In [16]:
eval_img.mean(0).max()

154.2666778564453

In [17]:
eval_img.mean()

117.89236935805668

In [18]:
preds_tmp, n_per_img, plot_data = recognition(model=chromo_model, eval_imgs_all=eval_img,
                                             batch_size=16, use_tqdm=False,
                                             nms=True, candidate_threshold=0.2,
                                             nms_threshold=0.6, 
                                             pixel_nm=chromo_model.data_generator.psf_params['pixel_size_xy'],
                                             plot_num=1,
                                             win_size=128,
                                             padding=True,
                                             start_field_pos=[0, 0],
                                             padded_background=chromo_model.evaluation_params['padded_background'])

processing area:1/99, input field_xy:[ -2 145  -3 144], use_coordconv:True, retain locs in area:[-2, 125, -3, 124]

/home/pk/Documents/cellbgnet/cellbgnet/utils/hardware.py:9: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  return torch.cuda.FloatTensor(x.astype('float32'))


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]


In [19]:
plot_full_img_predictions(chromo_model, plot_infs=plot_data, eval_csv=None, plot_num=1, fov_size=fov_size, pixel_size=[65, 65])

In [20]:
img_infs = assemble_full_img_predictions(chromo_model, plot_data)

In [21]:
img_infs.keys()

dict_keys(['Probs', 'XO', 'YO', 'ZO', 'Int', 'BG', 'XO_sig', 'YO_sig', 'ZO_sig', 'Int_sig', 'Probs_ps', 'XO_ps', 'YO_ps', 'ZO_ps', 'Samples_ps', 'raw_img', 'only_bg'])

In [22]:
for key, value in img_infs.items():
    plt.figure()
    plt.imshow(value[400: 700, 500:800])
    plt.colorbar()
    plt.title(key)
    plt.show()

In [24]:
z_values = np.array([x[4] for x in preds_tmp])
ph_values = np.array([x[5] for x in preds_tmp])

In [34]:
plt.figure()
plt.hist(z_values, bins=200)
plt.show()

#### Testing real image

In [26]:
images_dir = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4791/Run/pool_CB4791_pos76/venus/')
img_filenames = sorted(list(images_dir.glob('*.tiff')))

eval_imgs_all = []
for filename in img_filenames:
    eval_imgs_all.append(imread(filename))
eval_imgs_all = np.stack(eval_imgs_all[:])

In [27]:
eval_imgs_all.shape

(150, 1041, 1302)

In [28]:
#len(np.unique(mask_img))

In [29]:
preds_tmp, n_per_img, plot_data = recognition(model=chromo_model, eval_imgs_all=eval_imgs_all,
                                             batch_size=16, use_tqdm=False,
                                             nms=True, candidate_threshold=0.2,
                                             nms_threshold=0.5, 
                                             pixel_nm=chromo_model.data_generator.psf_params['pixel_size_xy'],
                                             plot_num=1,
                                             win_size=128,
                                             padding=True,
                                             start_field_pos=[0, 0],
                                             padded_background=chromo_model.evaluation_params['padded_background'])

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]


In [31]:
plot_full_img_predictions(chromo_model, plot_infs=plot_data, eval_csv=None, plot_num=1,
                        fov_size=fov_size, pixel_size=[65, 65])

In [32]:
len(preds_tmp)

85180

In [32]:
img_infs = assemble_full_img_predictions(chromo_model, plot_data)

In [33]:
img_infs.keys()

dict_keys(['Probs', 'XO', 'YO', 'ZO', 'Int', 'BG', 'XO_sig', 'YO_sig', 'ZO_sig', 'Int_sig', 'Probs_ps', 'XO_ps', 'YO_ps', 'ZO_ps', 'Samples_ps', 'raw_img', 'only_bg'])

In [34]:
for key, value in img_infs.items():
    plt.figure()
    plt.imshow(value[400: 700, 500:800])
    plt.colorbar()
    plt.title(key)
    plt.show()

In [35]:
plt.figure()
plt.imshow(mask_img[400: 700, 500:800])
plt.title('cell mask')
plt.show()

#### Predictions are a list of numbers where each element corresponds to a localizaiton in the following order


    1. counter of the molecule per tile.. not a global counter on the frame, need to fix this
    2. image number used to index into the number of the image in the prediction arrays.
    Eval image is of shape [1, 1041, 1302], so, the image number will always be 1, if you give one image at a time
    3. x position in nm where 0 is top left corner
    4. y position in nm where 0 is top left corner
    5. z position in nm where 0 is from the reference 0 nm in height
    6. photon counts 
    7. probability afer nms
    8. x_sigma in nm 
    9. y_sigma in nm
    10. z_sigma in nm
    11. photon_counts_sigma
    12. x offset
    13. y offset


In [21]:
preds_tmp

[[1.0,
  1.0,
  28321.5390625,
  6723.5126953125,
  231.839599609375,
  1255.1942138671875,
  0.6568737626075745,
  13.57247543334961,
  19.955181121826172,
  63.516578674316406,
  273.5525817871094,
  0.21598798036575317,
  -0.06134552136063576],
 [2.0,
  1.0,
  31802.8359375,
  6725.31005859375,
  413.4086608886719,
  2310.737060546875,
  0.6244803667068481,
  28.462249755859375,
  23.14813804626465,
  41.45981216430664,
  335.16925048828125,
  -0.22561213374137878,
  -0.033694442361593246],
 [3.0,
  1.0,
  29890.365234375,
  6743.890625,
  481.10693359375,
  1487.901123046875,
  0.24742525815963745,
  64.95372772216797,
  55.89762496948242,
  13.929397583007812,
  295.5343017578125,
  0.3517780005931854,
  -0.7478349208831787],
 [4.0,
  1.0,
  33122.10546875,
  6842.93896484375,
  386.67425537109375,
  1144.3037109375,
  0.2555111050605774,
  64.86029815673828,
  44.50193405151367,
  55.326412200927734,
  188.57318115234375,
  0.07084576785564423,
  -0.2240137755870819],
 [5.0,
  1.

In [23]:
img_infs = assemble_full_img_predictions(chromo_model, plot_data)

In [24]:
img_infs.keys()

dict_keys(['Probs', 'XO', 'YO', 'ZO', 'Int', 'BG', 'XO_sig', 'YO_sig', 'ZO_sig', 'Int_sig', 'Probs_ps', 'XO_ps', 'YO_ps', 'ZO_ps', 'Samples_ps', 'raw_img', 'only_bg'])

In [25]:
for key, value in img_infs.items():
    plt.figure()
    plt.imshow(value[400: 700, 500:800])
    plt.colorbar()
    plt.title(key)
    plt.show()

In [23]:
plt.figure()
plt.imshow(img_infs['BG'][400: 700, 500:800], vmin=0, vmax=300, cmap='gray')
plt.colorbar()
plt.title('Predicted PSF')
plt.show()

In [24]:
plt.figure()
plt.imshow(img_infs['raw_img'][400: 700, 500:800], vmin=25, vmax=520 , cmap='gray')
plt.colorbar()
plt.title('Raw image')
plt.show()

In [51]:
plt.figure()
plt.imshow(img_infs['only_bg'][:-200, :-200])
plt.colorbar()
plt.title('Only bg')
plt.show()

##### Collect predictions of all emitters in one image for plotting distributions

In [30]:
x_global = np.array([x[2] for x in preds_tmp])
y_global = np.array([x[3] for x in preds_tmp])
z_global = np.array([x[4] for x in preds_tmp])
ph_global = np.array([x[5] for x in preds_tmp])
prob_dot_nms = np.array([x[6] for x in preds_tmp])
sigma_x = np.array([x[7] for x in preds_tmp])
sigma_y = np.array([x[8] for x in preds_tmp])
sigma_z = np.array([x[9] for x in preds_tmp])
sigma_ph = np.array([x[10] for x in preds_tmp])

#### Only plot z, photon counts and probablitiy so that you can check what is going on as time goes by

In [32]:
fig, ax = plt.subplots(nrows=1, ncols=2)

ax[0].hist(z_global, bins=50)
ax[0].axvspan(z_global.min(), z_global.max(), color='green', alpha=0.1)
ax[0].set_xlabel('z [nm] of the predicted emitters')
ax[0].set_ylabel('counts')

ax[1].hist(ph_global, bins=50)
ax[1].axvspan(ph_global.min(), ph_global.max(), color='green', alpha=0.1)
ax[1].set_xlabel('photon counts of the predicted emitters')
ax[1].set_ylabel('counts')

#plt.suptitle(img_path)
plt.show()

### Plotting sigma-xyzi distributions of a single image (sigmas) predicted by the network

In [33]:

fig, ax = plt.subplots(nrows=2, ncols=2)
ax[0, 0].hist(sigma_x, bins=25)
ax[0, 0].axvspan(sigma_x.min(), sigma_x.max(), color='green', alpha=0.1)
ax[0, 0].set_xlabel(r'$\sigma_x$ [nm]')
ax[0, 0].set_ylabel('counts')
ax[0, 0].set_title(f"Mean sigma_x is {np.mean(sigma_x):0.3f}, min:{np.min(sigma_x):0.3f}, max:{np.max(sigma_x):0.3f}")

ax[0, 1].hist(sigma_y, bins=25)
ax[0, 1].axvspan(sigma_y.min(), sigma_y.max(), color='green', alpha=0.1)
ax[0, 1].set_xlabel(r'$\sigma_y$ [nm]')
ax[0, 1].set_ylabel('counts')
ax[0, 1].set_title(f"Mean sigma_y is {np.mean(sigma_y):0.3f}, min:{np.min(sigma_y):0.3f}, max:{np.max(sigma_y):0.3f}")


ax[1, 0].hist(sigma_z, bins=25)
ax[1, 0].axvspan(sigma_z.min(), sigma_z.max(), color='green', alpha=0.1)
ax[1, 0].set_xlabel(r'$\sigma_z$ [nm]')
ax[1, 0].set_ylabel('counts')
ax[1, 0].set_title(f"Mean sigma_z is {np.mean(sigma_z):0.3f}, min:{np.min(sigma_z):0.3f}, max:{np.max(sigma_z):0.3f}")

ax[1, 1].hist(sigma_ph, bins=25)
ax[1, 1].axvspan(sigma_ph.min(), sigma_ph.max(), color='green', alpha=0.1)
ax[1, 1].set_xlabel(r'$\sigma_{ph}$ [nm]')
ax[1, 1].set_ylabel('counts')
ax[1, 1].set_title(f"Mean sigma_ph is {np.mean(sigma_ph):0.3f}, min:{np.min(sigma_ph):0.3f}, max:{np.max(sigma_ph):0.3f}")

plt.show()

#### Plot distributions on predictions in one image xyzi

##### Loop over all images and plot variability in time

In [28]:
img_filenames

[PosixPath('/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000000.tiff'),
 PosixPath('/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000001.tiff'),
 PosixPath('/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000002.tiff'),
 PosixPath('/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000003.tiff'),
 PosixPath('/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000004.tiff'),
 PosixPath('/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000005.tiff'),
 PosixPath('/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000006.tiff'),
 PosixPath('/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000007.tiff'),
 PosixPath('/mnt/sda1/SMLAT/data/real_data/chromosome_do

In [29]:
number_preds = []
for img_path in img_filenames:
    real_img = imread(img_path)[np.newaxis, :]
    preds_tmp, n_per_img, plot_data = recognition(model=chromo_model, eval_imgs_all=real_img,
                                             batch_size=16, use_tqdm=False,
                                             nms=True, candidate_threshold=0.05,
                                             nms_threshold=0.05, 
                                             pixel_nm=chromo_model.data_generator.psf_params['pixel_size_xy'],
                                             plot_num=1,
                                             win_size=128,
                                             padding=True,
                                             start_field_pos=[0, 0],
                                             padded_background=chromo_model.evaluation_params['padded_background'])
    number_preds.append(len(preds_tmp))
    print(img_path)

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000000.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000001.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000002.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000003.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, ret

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000036.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000037.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000038.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000039.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, ret

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000072.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000073.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000074.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000075.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, ret

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000108.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000109.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000110.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000111.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, ret

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000144.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000145.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000146.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000147.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, ret

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000180.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000181.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000182.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000183.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, ret

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000216.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000217.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000218.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000219.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, ret

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000252.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000253.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000254.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000255.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, ret

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000288.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000289.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000290.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_midway_pos32/venus/Pos32_img_000000291.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, ret

In [32]:
plt.figure()
plt.plot(np.arange(0, len(number_preds)), number_preds)
plt.xlabel('Frame number')
plt.ylabel('Number of dots predicted')
plt.title('3041 midway')
plt.show()